In [8]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import math
import numpy as np

In [10]:
img1 = Image.open('image1.jpg')
plt.figure()
plt.imshow(img1)
print("Please click")
x = plt.ginput(3)
print("clicked", x)
plt.show()

Please click


/usr/lib/python3/dist-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


clicked [(643.62903225806451, 149.00000000000011), (383.5, 196.48387096774195), (575.5, 351.32258064516128)]


In [11]:
img2 = Image.open('image2.jpg')
plt.figure()
plt.imshow(img2)
print("Please click")
y = plt.ginput(3)
print("clicked", y)
plt.show()

Please click


/usr/lib/python3/dist-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


clicked [(517.69354838709683, 435.96774193548384), (391.75806451612902, 266.67741935483878), (666.33870967741939, 138.67741935483878)]


In [12]:
print ("points in x",x)
print("points in y",y)

points in x [(643.62903225806451, 149.00000000000011), (383.5, 196.48387096774195), (575.5, 351.32258064516128)]
points in y [(517.69354838709683, 435.96774193548384), (391.75806451612902, 266.67741935483878), (666.33870967741939, 138.67741935483878)]
